<a href="https://colab.research.google.com/github/hikmatfarhat-ndu/pytorch/blob/main/custom_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

In [81]:
!git clone https://github.com/emanhamed/Houses-dataset

fatal: destination path 'Houses-dataset' already exists and is not an empty directory.


In [12]:
df=pd.read_csv("Houses-dataset/Houses Dataset/HousesInfo.txt",header=None,delim_whitespace=True,
               names=["bedrooms","bathrooms","size","zipcode","price"])

In [44]:
x=df['zipcode'].value_counts().keys()

In [62]:
x=df['zipcode']==92276

In [76]:
y=df[x]

In [77]:
y.index

Int64Index([285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 330, 331, 332,
            333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345,
            346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358,
            359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371,
            372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384,
            385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 396, 397, 398,
            399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411,
            412, 413, 414, 415, 416, 417, 418, 419, 420],
           dtype='int64')

In [95]:
class CustomDataset(Dataset):
  def __init__(self,csvFile,imgDir):
    self.housesInfo=pd.read_csv(csvFile,header=None,delim_whitespace=True,
               names=["bedrooms","bathrooms","size","zipcode","price"])
    self.features=self.cleanData(self.housesInfo).to_numpy(np.float32)

  def cleanData(self,df):
    # compute the number of entries per zipcode
    zipcodes=df['zipcode'].value_counts().keys().tolist()
    counts=df['zipcode'].value_counts().tolist()
    #discard all zipcodes ocurring less than 20 times
    for count,zipcode in zip(counts,zipcodes):
      if count<20:
        idx=df[df['zipcode']==zipcode].index
        df.drop(idx,inplace=True)
    return df
  def getValues(self):
    return self.features


    

In [96]:
dataset=CustomDataset("Houses-dataset/Houses Dataset/HousesInfo.txt","Houses-dataset/Houses Dataset/")

In [97]:
type(dataset)

__main__.CustomDataset

In [98]:
data=dataset.getValues()

In [99]:
type(data)

numpy.ndarray

In [100]:
data.shape

(384, 5)

In [101]:
type(data[0,0])

numpy.float32

In [102]:
data[0]

array([5.0000e+00, 3.0000e+00, 2.5200e+03, 9.3446e+04, 7.8900e+05],
      dtype=float32)

In [1]:
zip=data[:,3]

NameError: ignored